In [1]:
import pandas as pd
import numpy as np

In [ ]:
def reverse_compliment(seq):
    translateTable = seq.maketrans("ATCG", "TAGC")
    output = seq.translate(translateTable)
    output = output[::-1]
    return(output)

In [2]:
linker_head = 'AGTCCTCTAAG'
actual_HIV = 'AGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCA'
n10 = 'TCTGGTAACT'

In [25]:
# read from the file
# remove all canonical entries (containing 'TCTCTAGCA\t')
# file = open("1TAGNGAGT2GCGCGAGA.txt") # WT
file = open("1TAGNGAGT2CTATCGCT.txt") # G19S
lines = []
canonical = 0
for line in file.readlines():
    if 'TCTCTAGCA\t' not in line:
        lines.append(line)
    else:
        canonical += 1

print('# aberrant LTR: ' + str(len(lines)))
print('# canonical LTR: ' + str(canonical))

787991
107531


In [26]:
# extracte the read 1 sequence from the spreadsheet
reads = []
for line in lines:
    read = line.split('\t')[2]
    reads.append(read)

In [27]:
# trim the (primer), n10, and linker
trimmed_reads = []
missing_n10_count = 0
for idx, read in enumerate(reads):
    if n10 in read:
        read = (read.split(linker_head)[0]).split(n10)[1]
        trimmed_reads.append(read)
    else:
        reads.pop(idx)
        missing_n10_count += 1 # count incomplete n10

print('# aberrant LTRs with n10: ' + str(len(trimmed_reads)))

602529


In [28]:
# base-by-base comparison with true HIV sequence
aberrant = []
for read in trimmed_reads:
    human_start_pos = next((i for i, (char1, char2) in enumerate(zip(read, actual_HIV)) if char1 != char2), None)
    if human_start_pos != None:
        human = read[human_start_pos:]
        aberrant.append(human)
    # aberrant.append(human_start_pos)
print(len(aberrant))
#TO DO: 

103900


In [29]:
from collections import Counter 

aberrrant_occur = Counter(aberrant)
aberrrant_occur.most_common(20)

[('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGT', 1222),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAG', 1194),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTA', 751),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTAC', 663),
 ('AATTTGGTAGTGTGGAAAATCTCTAGCAAAGGGCGAATTCTGCAGATATCCATCACACTGGCGGCCGCTCGAGCATGCATCTAG',
  558),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCT', 548),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTA', 547),
 ('ATAGCAG', 527),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTT', 474),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATC', 462),
 ('A', 435),
 ('G', 422),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACCAGAG', 386),
 ('ATAGCAGT', 383),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACC', 340),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACCA', 300),
 ('AATTTGGTAGTGTGGAAAATC', 264),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACCAGAGTC', 260),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACCAGA', 250),
 ('TTTCCACACTGACTAAAAGGGTCTGAGGGATCTCTAGTTACCAG', 246)]

In [16]:
file_new = open('aberrant_LTR_01_all.fa', 'w')
# file_new = open('aberrant_LTR_02_all.fa', 'w')
c = 0
for x in aberrant:
	file_new.write('>seq_' + str(c) + '\n')
	file_new.write(str(x) + '\n')
	c += 1
file_new.close()

In [30]:
file_new = open('aberrant_LTR_01_unique.fa', 'w')
# file_new = open('aberrant_LTR_02_unique.fa', 'w')
c = 0
for x in list(aberrrant_occur.keys()): # TODO: sort
	file_new.write('>seq_' + str(c) + '\n') # TODO: add count
	file_new.write(str(x) + '\n')
	c += 1
file_new.close()